In [1]:
# What version of Python do you have?
import sys
import os
import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import numpy as np
import re
import mne
import pathlib
import openpyxl
from datetime import datetime
import pytz
import matplotlib
import random
import os
from skimage.restoration import (denoise_wavelet, estimate_sigma)
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, Sequential
from matplotlib import pyplot as plt
from scipy import signal
plt.ioff()
import psutil
import gc
import glob
pd.io.parquet.get_engine('auto').__class__
%matplotlib inline
matplotlib.use('Qt5Agg')
#matplotlib.use('agg')

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Init Plugin
Init Graph Optimizer
Init Kernel
Tensor Flow Version: 2.5.0
Keras Version: 2.5.0

Python 3.9.7 (default, Sep 16 2021, 23:53:23) 
[Clang 12.0.0 ]
Pandas 1.3.1
Scikit-Learn 0.24.2
GPU is available


In [2]:
cwd = os.getcwd()
FREQ = 500
database_path = '../../Dataset/EMU_monitor(ruc)/'
save_csv_path = "../DataSet/CHB-MIT/filtered_df_csv"
edf_file_type = ".edf"
capitilize_edf_file_type = ".EDF"
bdf_file_type = ".BDF"
patient_one_path = 'chb04/'
info_df_path = "../../Dataset/EMU_monitor(ruc)/Eventlist_RUC.xlsx"
external_hardisk_drive_path = os.path.dirname('/Volumes/LaCie/Database/')
cwd

'/Users/niklashjort/Desktop/Notes/Speciale/projects/DataHandling/Køge'

In [3]:
def ReadEdfFile(file_name, print_reader_info = False):
    if edf_file_type in file_name or capitilize_edf_file_type in file_name:
        if(print_reader_info):
            data = mne.io.read_raw_edf(file_name)
            raw_data = data.get_data()
            converted_raw = pd.DataFrame(raw_data.transpose(), columns=data.ch_names)
            print(data.info)
            return converted_raw
        else:
            data = mne.io.read_raw_edf(file_name, verbose='error')
            raw_data = data.get_data()
            converted_raw = pd.DataFrame(raw_data.transpose(), columns=data.ch_names)
            return converted_raw
    if bdf_file_type in file_name:
        if(print_reader_info):
            data = mne.io.read_raw_bdf(file_name)
            raw_data = data.get_data()
            converted_raw = pd.DataFrame(raw_data.transpose(), columns=data.ch_names)
            print(data.info)
            return converted_raw
        else:
            data = mne.io.read_raw_edf(file_name, verbose='error')
            raw_data = data.get_data()
            converted_raw = pd.DataFrame(raw_data.transpose(), columns=data.ch_names)
            return converted_raw

In [4]:
def get_file_t_start_ms(filename):
    str_t_start = re.match(r".*Patient [0-9]+/(.*?)_", filename)
    if str_t_start:
        str_t_start = str_t_start.group(1)
        t_start = datetime.strptime(str_t_start, '%Y%m%d%H%M%S')
        return t_start.timestamp() * 1000
    else:
        str_t_start = "Time start is not present"
        return str_t_start
    

In [5]:
def insert_time_stamp(dataframe, file_start_time, frq=FREQ):
    period_row_increment_value =  (1 / frq) * 1000
    dataframe.insert(0, "timestamp", [file_start_time + i * period_row_increment_value for i in dataframe.index])

In [14]:
# for c in info_list[0:1]:
#     patient_id = c['ID']
#     file_name = c['File']
#     file_t_start = get_file_t_start_ms(file_name)
#     print(file_name)
#     print(file_t_start)
#     sz_info_list = []
#     current_file_info = info_df.loc[(info_df['patientID'] == patient_id) & (info_df['fullPath'] == file_name)]
    

../../Dataset/EMU_monitor(ruc)/EEG/Patient 4/20190521155643_p4Nat2.sdeeg_converted_.easy_filtered.edf
1558447003000.0


In [14]:
#tester = ReadEdfFile("../../Dataset/EMU_monitor(ruc)/EKG/Patient 21/10-39-08.EDF", True)

Extracting EDF parameters from /Users/niklashjort/Desktop/Notes/Speciale/projects/Dataset/EMU_monitor(ruc)/EKG/Patient 21/10-39-08.EDF...
EDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
ECG, Accelerometer_X, Accelerometer_Y, Accelerometer_Z, Marker, HRV
Creating raw.info structure...
<Info | 7 non-empty values
 bads: []
 ch_names: ECG, Accelerometer_X, Accelerometer_Y, Accelerometer_Z, Marker, HRV
 chs: 6 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 250.0 Hz
 meas_date: 2020-06-15 10:39:08 UTC
 nchan: 6
 projs: []
 sfreq: 500.0 Hz
>


In [6]:
def test(time_emu):
    sz = datetime.strptime(time_emu, '%d-%m-%Y %H:%M:%S')
    return sz

def test2(time_emu):
    time = datetime.strptime(time_emu, '%Y-%m-%d %H:%M:%S')
    return time

In [5]:
def read_info(filename):
    x = mne.io.read_raw_edf(filename)
    info = x.info
    raw_data = x.get_data()
    print(info)
    converted_raw = pd.DataFrame(raw_data.transpose(), columns=x.ch_names)
    return converted_raw, str(info['meas_date']), str(info['sfreq'])

In [8]:
sz_start = test("22-05-2019 02:45:22")
sz_start

datetime.datetime(2019, 5, 22, 2, 45, 22)

In [6]:
test_df, file_start, freq = read_info("/Users/niklashjort/Desktop/Notes/Speciale/projects/Dataset/EMU_monitor(ruc)/EKG/Patient 6/11-00-04.EDF")
file_start

Extracting EDF parameters from /Users/niklashjort/Desktop/Notes/Speciale/projects/Dataset/EMU_monitor(ruc)/EKG/Patient 6/11-00-04.EDF...
EDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
ECG, Accelerometer_X, Accelerometer_Y, Accelerometer_Z, Marker, HRV
Creating raw.info structure...
<Info | 7 non-empty values
 bads: []
 ch_names: ECG, Accelerometer_X, Accelerometer_Y, Accelerometer_Z, Marker, HRV
 chs: 6 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 250.0 Hz
 meas_date: 2019-08-26 11:00:04 UTC
 nchan: 6
 projs: []
 sfreq: 500.0 Hz
>


'2019-08-26 11:00:04+00:00'

In [10]:
gc.collect()

0

In [11]:
freq = int(float(freq))
freq

500

In [12]:
file_start = test2("2020-06-15 10:39:08")
file_start

datetime.datetime(2020, 6, 15, 10, 39, 8)

In [ ]:
timestamped_sz_start = sz_start.timestamp() * 1000

In [13]:
timestamped_file_start = file_start.timestamp() * 1000
insert_time_stamp(test_df, timestamped_file_start, freq)

In [14]:
end_file_timestamp = test_df['timestamp'].iloc[-1]

In [15]:
datetime.fromtimestamp(end_file_timestamp/1000)

datetime.datetime(2020, 6, 18, 9, 44, 26, 998000)

In [29]:
timestamped_file_start < timestamped_sz_start < end_file_timestamp

True

In [30]:
gc.collect()

30